In [ ]:
dataColumnName = 'valueStdScaled'
model_exp = 'VARMA'

In [ ]:
if 'T' not in globals():
    T = 15
if 'predict_ahead' not in globals():
    predict_ahead = 15

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# load datasets
data = pd.read_csv('../../data/rq_1-3_train_test/2024-01-15_10-51-45__2024-01-15_14-26-45_load-gen-msg-w-spikes-10s-rate.csv', 
                   index_col=['EventDateTime'], parse_dates=['EventDateTime'])
dataLatency = pd.read_csv('../../data/rq_1-3_train_test/2024-01-15_10-51-45__2024-01-15_14-26-45_load-gen-avg-latency-10s-rate.csv', 
                          index_col='EventDateTime', parse_dates=['EventDateTime'])

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
%run '../lib/prepareDataSet.ipynb'

In [ ]:
%run '../lib/utils_anomaly_detection.ipynb'

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib notebook
%matplotlib inline

In [ ]:
import statsmodels.api as sm
from itertools import product

In [ ]:
#from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, plot_predict
#from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error

In [ ]:
X_test = dataFrame_test.to_frame()

In [ ]:
X_test['TimeLatencyStdScaled'] = dataFrameLatency_test

In [ ]:
X_test.head()

In [ ]:
X_test.shape

In [ ]:
cols = ['valueStdScaled', 'TimeLatencyStdScaled']

In [ ]:
X_train = dataFrame_train.to_frame()
X_train['TimeLatencyStdScaled'] = dataFrameLatency_train

In [ ]:
X_train.head()

In [ ]:
# VAR requires 2 time series. We have both response time and throughput available
# We also set the maxlags with the same value we use for the neural nets, namely T. 
# This means it will use the previous T values to predicts the next one.
# This is useful as we always have the real value (the T+1th value) so we can compare it with the prediction

In [ ]:
%%time
model_VAR = VAR(X_train)

In [ ]:
lag_order_results = model_VAR.select_order(maxlags=T)
lag_order_results

In [ ]:
lag_order_results.selected_orders

In [ ]:
results_VAR = model_VAR.fit(maxlags=T)

In [ ]:
lag_order = results_VAR.k_ar
lag_order # ensuring the maxlags is unchanged after training

In [ ]:
# This function generates the forecasts values for the test dataset, one step at a time
def generate_forecasts(x_test, model_VAR, T):
    forecasts_VAR = []
    for i in range(x_test.shape[0]-T+1): # we can't go past T and we start predicting the T+1 value
        prior = x_test[i:i+T][['valueStdScaled','TimeLatencyStdScaled']].to_numpy()
        fcast_VAR = model_VAR.forecast(prior, 1)
        forecasts_VAR.append(fcast_VAR)
        i += 1
    
    return forecasts_VAR

In [ ]:
# This function generates the forecasts values for the test dataset
# It predicts predict_ahead steps before getting a new ground truth value and predicts the next batch
def generate_nsteps_forecasts(x_test, model_VAR, T, predict_ahead):    
    y_predict = []
    index = 0
    while (index < x_test.shape[0]-T+1):
    #while len(y_predict) < x_test.shape[0]-T:
        #print(f'Selecting input from index {index} to {index+T}')
        last_x = x_test[index: index+T]
        p = model_VAR.forecast(last_x, predict_ahead)
        y_predict.append(p)
        index += predict_ahead

    y_pred_conc = y_predict[0]
    i=1
    while i < len(y_predict):
        y_pred_conc = np.concatenate((y_pred_conc, y_predict[i]), axis=0)
        i+=1

    return y_pred_conc

In [ ]:
X_train_valueStdScaled = []
Y_train_valueStdScaled = []

(X_train_valueStdScaled, Y_train_valueStdScaled) = formatData(X_train_valueStdScaled, Y_train_valueStdScaled, 
                                                          X_train, T, 2)

In [ ]:
X_train_valueStdScaled.shape, Y_train_valueStdScaled.shape

In [ ]:
X_test_valueStdScaled = []
Y_test_valueStdScaled = []

(X_test_valueStdScaled, Y_test_valueStdScaled) = formatData(X_test_valueStdScaled, Y_test_valueStdScaled, 
                                                          X_test, T, 2)

In [ ]:
Y_test_valueStdScaled.shape, X_test_valueStdScaled.shape

In [ ]:
X_test_valueStdScaled[0]

In [ ]:
y_pred_train = generate_forecasts(X_train, results_VAR, T)

In [ ]:
len(y_pred_train)

In [ ]:
y_pred_train = y_pred_train[0:X_train_valueStdScaled.shape[0]]

In [ ]:
len(y_pred_train)

In [ ]:
Y_train_pred = np.array(y_pred_train).reshape(-1,2)
YY_train_pred = pd.DataFrame(Y_train_pred, columns=['valueStdScaled', 'TimeLatencyStdScaled'])

In [ ]:
YY_train_pred.shape, Y_train_valueStdScaled.shape

In [ ]:
y_pred = generate_forecasts(X_test, results_VAR, T)

In [ ]:
y_pred_n_steps = generate_nsteps_forecasts(X_test.to_numpy(), results_VAR, T, predict_ahead)

In [ ]:
len(y_pred), len(y_pred_n_steps)

In [ ]:
y_pred = y_pred[0:X_test_valueStdScaled.shape[0]]

In [ ]:
y_pred_n_steps = y_pred_n_steps[0:X_test_valueStdScaled.shape[0]]

In [ ]:
Y_test_pred1step = np.array(y_pred).reshape(-1,2)
Y_test_pred_1_step = pd.DataFrame(Y_test_pred1step, columns=['valueStdScaled', 'TimeLatencyStdScaled'])

In [ ]:
Y_test_prednsteps = np.array(y_pred_n_steps).reshape(-1,2)
Y_test_pred_n_steps = pd.DataFrame(Y_test_prednsteps, columns=['valueStdScaled', 'TimeLatencyStdScaled'])

In [ ]:
Y_test_pred_n_steps.shape, Y_test_pred_1_step.shape, Y_test_valueStdScaled.shape

In [ ]:
#def expand_dataframe_with_nsteps(expand_df, steps):
#    for i in range(steps):
#        new_row = []
#        new_row.insert(0,{'valueStdScaled': expand_df['valueStdScaled'][0], 
#                          'TimeLatencyStdScaled': expand_df['TimeLatencyStdScaled'][0]})
#        nr = pd.DataFrame(new_row)
#        expand_df = pd.concat([nr, expand_df], ignore_index=True)  
#    
#    return expand_df

In [ ]:
Y_train_arr = np.array(Y_train_valueStdScaled).reshape(-1,2)
Y_train_frame = pd.DataFrame(Y_train_arr, columns=['valueStdScaled', 'TimeLatencyStdScaled'])

In [ ]:
Y_test_arr = np.array(Y_test_valueStdScaled).reshape(-1,2)
Y_test_frame = pd.DataFrame(Y_test_arr, columns=['valueStdScaled', 'TimeLatencyStdScaled'])

In [ ]:
errors_ae_train_value = calculate_absolute_prediction_errors(Y_train_frame['valueStdScaled'].to_numpy(), YY_train_pred['valueStdScaled'].to_numpy())
lo_3sigma_ae_value, up_3sigma_ae_value = calculate_3sigma_threshold(errors_ae_train_value)
anomalies_ae_train_value = calculate_3sigma_anomalies(errors_ae_train_value, lo_3sigma_ae_value, up_3sigma_ae_value)

errors_se_train_value = calculate_squared_prediction_errors(Y_train_frame['valueStdScaled'].to_numpy(), YY_train_pred['valueStdScaled'].to_numpy())
lo_3sigma_se_value, up_3sigma_se_value = calculate_3sigma_threshold(errors_se_train_value)
anomalies_se_train_value = calculate_3sigma_anomalies(errors_se_train_value, lo_3sigma_se_value, up_3sigma_se_value)


In [ ]:
errors_ae_train_time = calculate_absolute_prediction_errors(Y_train_frame['TimeLatencyStdScaled'].to_numpy(), YY_train_pred['TimeLatencyStdScaled'].to_numpy())
lo_3sigma_ae_time, up_3sigma_ae_time = calculate_3sigma_threshold(errors_ae_train_time)
anomalies_ae_train_time = calculate_3sigma_anomalies(errors_ae_train_time, lo_3sigma_ae_time, up_3sigma_ae_time)

errors_se_train_time = calculate_squared_prediction_errors(Y_train_frame['TimeLatencyStdScaled'].to_numpy(), YY_train_pred['TimeLatencyStdScaled'].to_numpy())
lo_3sigma_se_time, up_3sigma_se_time = calculate_3sigma_threshold(errors_se_train_time)
anomalies_se_train_time = calculate_3sigma_anomalies(errors_se_train_time, lo_3sigma_se_time, up_3sigma_se_time)

In [ ]:
errors_ae_test_value = calculate_absolute_prediction_errors(Y_test_frame['valueStdScaled'].to_numpy(), Y_test_pred_1_step['valueStdScaled'].to_numpy())
anomalies_ae_test_value = calculate_3sigma_anomalies(errors_ae_test_value, lo_3sigma_ae_value, up_3sigma_ae_value)

In [ ]:
errors_se_test_value = calculate_squared_prediction_errors(Y_test_frame['valueStdScaled'].to_numpy(), Y_test_pred_1_step['valueStdScaled'].to_numpy())
anomalies_se_test_value = calculate_3sigma_anomalies(errors_se_test_value, lo_3sigma_se_value, up_3sigma_se_value)

In [ ]:
errors_ae_test_time = calculate_absolute_prediction_errors(Y_test_frame['TimeLatencyStdScaled'].to_numpy(), Y_test_pred_1_step['TimeLatencyStdScaled'].to_numpy())
anomalies_ae_test_time = calculate_3sigma_anomalies(errors_ae_test_time, lo_3sigma_ae_time, up_3sigma_ae_time)

In [ ]:
errors_se_test_time = calculate_squared_prediction_errors(Y_test_frame['TimeLatencyStdScaled'].to_numpy(), Y_test_pred_1_step['TimeLatencyStdScaled'].to_numpy())
anomalies_se_test_time = calculate_3sigma_anomalies(errors_se_test_time, lo_3sigma_se_time, up_3sigma_se_time)

In [ ]:
fig = plt.figure(figsize=(20,15))
plt.title("Predict Spikes 1-step Y_test Value")
plt.plot(Y_test_frame['valueStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='gray')
plt.plot(Y_test_pred_1_step['valueStdScaled'].to_numpy(),label="Predict 1 step", alpha=0.6, c='red')
plt.scatter(np.where(anomalies_ae_test_value==True)[0], Y_test_frame['valueStdScaled'].to_numpy()[np.where(anomalies_ae_test_value==1)], 
            alpha=0.8, color='green', s=300, label="3-Sigma Anomalies AE")
plt.scatter(np.where(anomalies_se_test_value==True)[0], Y_test_frame['valueStdScaled'].to_numpy()[np.where(anomalies_se_test_value==1)], 
            alpha=0.8, color='magenta', s=150, label="3-Sigma Anomalies SE")
plt.legend()
figName = f"Y_test_value_1-step_spikes-T_{T}-StdScaled.png"
#mlflow.log_figure(fig, figName)
plt.savefig(figName, transparent=False)
#fig.clf()
#plt.close()

In [ ]:
fig = plt.figure(figsize=(20,15))
plt.title("Predict Spikes 1-step Y_test TimeLatency")
plt.plot(Y_test_frame['TimeLatencyStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='gray')
plt.plot(Y_test_pred_1_step['TimeLatencyStdScaled'].to_numpy(),label="Predict 1 step", alpha=0.6, c='red')
plt.scatter(np.where(anomalies_ae_test_time==True)[0], Y_test_frame['TimeLatencyStdScaled'].to_numpy()[np.where(anomalies_ae_test_time==1)], 
            alpha=0.8, color='green', s=300, label="3-Sigma Anomalies AE")
plt.scatter(np.where(anomalies_se_test_time==True)[0], Y_test_frame['TimeLatencyStdScaled'].to_numpy()[np.where(anomalies_se_test_time==1)], 
            alpha=0.8, color='magenta', s=150, label="3-Sigma Anomalies SE")
plt.legend()
figName = f"Y_test_time_1-step_spikes-T_{T}-StdScaled.png"
#mlflow.log_figure(fig, figName)
plt.savefig(figName, transparent=False)
#fig.clf()
#plt.close()

In [ ]:
errors_ae_test_value_nstep = calculate_absolute_prediction_errors(Y_test_frame['valueStdScaled'].to_numpy(), Y_test_pred_n_steps['valueStdScaled'].to_numpy())
anomalies_ae_test_value_nstep = calculate_3sigma_anomalies(errors_ae_test_value_nstep, lo_3sigma_ae_value, up_3sigma_ae_value)

In [ ]:
errors_se_test_value_nstep = calculate_squared_prediction_errors(Y_test_frame['valueStdScaled'].to_numpy(), Y_test_pred_n_steps['valueStdScaled'].to_numpy())
anomalies_se_test_value_n_step = calculate_3sigma_anomalies(errors_se_test_value_nstep, lo_3sigma_se_value, up_3sigma_se_value)

In [ ]:
errors_ae_test_time_nstep = calculate_absolute_prediction_errors(Y_test_frame['TimeLatencyStdScaled'].to_numpy(), Y_test_pred_n_steps['TimeLatencyStdScaled'].to_numpy())
anomalies_ae_test_time_nstep = calculate_3sigma_anomalies(errors_ae_test_time_nstep, lo_3sigma_ae_time, up_3sigma_ae_time)

In [ ]:
errors_se_test_time_nstep = calculate_squared_prediction_errors(Y_test_frame['TimeLatencyStdScaled'].to_numpy(), Y_test_pred_n_steps['TimeLatencyStdScaled'].to_numpy())
anomalies_se_test_time_nstep = calculate_3sigma_anomalies(errors_se_test_time_nstep, lo_3sigma_se_time, up_3sigma_se_time)

In [ ]:
fig = plt.figure(figsize=(20,15))
plt.title(f"Predict Spikes {predict_ahead}-steps Y_test Value")
plt.plot(Y_test_frame['valueStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='gray')
plt.plot(Y_test_pred_n_steps['valueStdScaled'].to_numpy(),label="Predict n_step", alpha=0.6, c='red')
plt.scatter(np.where(anomalies_ae_test_value_nstep==True)[0], Y_test_frame['valueStdScaled'].to_numpy()[np.where(anomalies_ae_test_value_nstep==1)], 
            alpha=0.8, color='green', s=300, label="3-Sigma Anomalies AE")
plt.scatter(np.where(anomalies_se_test_value_n_step==True)[0], Y_test_frame['valueStdScaled'].to_numpy()[np.where(anomalies_se_test_value_n_step==1)], 
            alpha=0.8, color='magenta', s=150, label="3-Sigma Anomalies SE")
plt.legend()
figName = f"Y_test_value_{predict_ahead}-step_spikes-T_{T}-StdScaled.png"
#mlflow.log_figure(fig, figName)
plt.savefig(figName, transparent=False)
#fig.clf()
#plt.close()

In [ ]:
fig = plt.figure(figsize=(20,15))
plt.title(f"Predict Spikes {predict_ahead}-steps Y_test TimeLatency")
plt.plot(Y_test_frame['TimeLatencyStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='gray')
plt.plot(Y_test_pred_n_steps['TimeLatencyStdScaled'].to_numpy(),label="Predict n_step", alpha=0.6, c='red')
plt.scatter(np.where(anomalies_ae_test_time_nstep==True)[0], Y_test_frame['TimeLatencyStdScaled'].to_numpy()[np.where(anomalies_ae_test_time_nstep==1)], 
            alpha=0.8, color='green', s=300, label="3-Sigma Anomalies AE")
plt.scatter(np.where(anomalies_se_test_time_nstep==True)[0], Y_test_frame['TimeLatencyStdScaled'].to_numpy()[np.where(anomalies_se_test_time_nstep==1)], 
            alpha=0.8, color='magenta', s=150, label="3-Sigma Anomalies SE")
plt.legend()
figName = f"Y_test_time_{predict_ahead}-step_spikes-T_{T}-StdScaled.png"
#mlflow.log_figure(fig, figName)
plt.savefig(figName, transparent=False)
#fig.clf()
#plt.close()

In [ ]:
aic = calculate_aic(len(Y_test_frame['TimeLatencyStdScaled']), 
                    mean_squared_error(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_1_step['TimeLatencyStdScaled']),
                    2
                   )

print(f"Calculating scores for TimeLatencyStdScaled with forecast T={T}, predict_ahead=1\n"
      f" R2: {r2_score(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_1_step['TimeLatencyStdScaled'])} \n"
      f"MAE: {mean_absolute_error(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_1_step['TimeLatencyStdScaled'])}\n"
      f"MAPE: {mean_absolute_percentage_error(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_1_step['TimeLatencyStdScaled'])}\n"
      f"MSE: {mean_squared_error(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_1_step['TimeLatencyStdScaled'])}\n"
      f"Pearson correlation: {np.corrcoef(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_1_step['TimeLatencyStdScaled'])[0,1]}\n"
      f"AIC: {aic}\n"      
     )

In [ ]:
aic = calculate_aic(len(Y_test_frame['valueStdScaled']), 
                    mean_squared_error(Y_test_frame['valueStdScaled'], Y_test_pred_n_steps['valueStdScaled']),
                    2
                   )

print(f"Calculating scores for valueStdScaled with forecast T={T}, predict_ahead=1\n"
      f" R2: {r2_score(Y_test_frame['valueStdScaled'], Y_test_pred_1_step['valueStdScaled'])} \n"
      f"MAE: {mean_absolute_error(Y_test_frame['valueStdScaled'], Y_test_pred_1_step['valueStdScaled'])}\n"
      f"MAPE: {mean_absolute_percentage_error(Y_test_frame['valueStdScaled'], Y_test_pred_1_step['valueStdScaled'])}\n"
      f"MSE: {mean_squared_error(Y_test_frame['valueStdScaled'], Y_test_pred_1_step['valueStdScaled'])}\n"
      f"Pearson correlation: {np.corrcoef(Y_test_frame['valueStdScaled'], Y_test_pred_1_step['valueStdScaled'])[0,1]}\n"
      f"AIC: {aic}\n"      
     )

In [ ]:
aic = calculate_aic(len(Y_test_frame['TimeLatencyStdScaled']), 
                    mean_squared_error(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_n_steps['TimeLatencyStdScaled']),
                    2
                   )

print(f"Calculating scores for TimeLatencyStdScaled with forecast T={T}, predict_ahead={predict_ahead}\n"
      f" R2: {r2_score(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_n_steps['TimeLatencyStdScaled'])} \n"
      f"MAE: {mean_absolute_error(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_n_steps['TimeLatencyStdScaled'])}\n"
      f"MAPE: {mean_absolute_percentage_error(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_n_steps['TimeLatencyStdScaled'])}\n"
      f"MSE: {mean_squared_error(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_n_steps['TimeLatencyStdScaled'])}\n"
      f"Pearson correlation: {np.corrcoef(Y_test_frame['TimeLatencyStdScaled'], Y_test_pred_n_steps['TimeLatencyStdScaled'])[0,1]}\n"
      f"AIC: {aic}\n"      
     )

In [ ]:
aic = calculate_aic(len(Y_test_frame['valueStdScaled']), 
                    mean_squared_error(Y_test_frame['valueStdScaled'], Y_test_pred_n_steps['valueStdScaled']),
                    2
                   )
print(f"Calculating scores for valueStdScaled with forecast T={T}, predict_ahead={predict_ahead}\n"
      f" R2: {r2_score(Y_test_frame['valueStdScaled'], Y_test_pred_n_steps['valueStdScaled'])} \n"
      f"MAE: {mean_absolute_error(Y_test_frame['valueStdScaled'], Y_test_pred_n_steps['valueStdScaled'])}\n"
      f"MAPE: {mean_absolute_percentage_error(Y_test_frame['valueStdScaled'], Y_test_pred_n_steps['valueStdScaled'])}\n"
      f"MSE: {mean_squared_error(Y_test_frame['valueStdScaled'], Y_test_pred_n_steps['valueStdScaled'])}\n"
      f"Pearson correlation: {np.corrcoef(Y_test_frame['valueStdScaled'], Y_test_pred_n_steps['valueStdScaled'])[0,1]}\n"
      f"AIC: {aic}\n"
     )

In [ ]:
results_VAR.save('results_VAR_stdScaled.pkl')

In [ ]:
from pathlib import Path
import os, sys

dataFileList = !ls ../../data/rq2-valid/*msg-w-spikes*.csv
dataLatencyFileList =  !ls ../../data/rq2-valid/*avg-latency*.csv

In [ ]:
### Update the below on the latest from above calculations on 3-Sigma

In [ ]:
results = {}
for filePos in range(len(dataFileList)):
    
    data = pd.read_csv(dataFileList[filePos], index_col='EventDateTime', parse_dates=['EventDateTime'])
    dataLatency = pd.read_csv(dataLatencyFileList[filePos], index_col='EventDateTime', parse_dates=['EventDateTime'])
    print(f'Processing files at position {filePos} in list')
    %run '../lib/prepareDataSet.ipynb'
    #cols = ['valueStdScaled', 'TimeLatencyStdScaled']
    trial_fname = os.path.basename(dataFileList[filePos])

    X_test = dataFrame_test.to_frame()
    X_test['TimeLatencyStdScaled'] = dataFrameLatency_test

    y_pred = generate_forecasts(X_test, results_VAR, T)
    X = np.array(y_pred).reshape(-1,2)
    XX = pd.DataFrame(X, columns=['valueStdScaled', 'TimeLatencyStdScaled'])
    XX_new = None
    shape_dif = X_test.shape[0] - XX.shape[0] 
    if shape_dif > 0:
        XX_new = expand_dataframe_with_nsteps(XX, shape_dif)
    elif shape_dif < 0:
        XX_new = XX2[0:X_test.shape[0]]
    else:
        XX_new = XX.copy()

    XX = XX_new.copy()
    XX.set_index(X_test.index, inplace=True)
    # These are the predictions one step at a time always looking at the ground truth
    X_test['valueStdScaledVarForecast'] = XX['valueStdScaled']
    X_test['TimeLatencyStdScaledVarForecast'] = XX['TimeLatencyStdScaled']    

    errors_ae = calculate_absolute_prediction_errors(X_test['valueStdScaled'].to_numpy(), X_test['valueStdScaledVarForecast'].to_numpy())
    anomalies_ae = calculate_3sigma_anomalies(errors_ae)
    errors_se = calculate_squared_prediction_errors(X_test['valueStdScaled'].to_numpy(), X_test['valueStdScaledVarForecast'].to_numpy())
    anomalies_se = calculate_3sigma_anomalies(errors_se)
    
    anomalies_3sigma_Y_test = calculate_3sigma_anomalies(X_test['valueStdScaled'].to_numpy())
    anomalies_3sigma_y_predict = calculate_3sigma_anomalies(X_test['valueStdScaledVarForecast'].to_numpy())
    
    anomalies_Y_test, z_scores_Y_test = calculate_zscore_anomalies(X_test['valueStdScaled'].to_numpy())
    anomalies_y_predict, z_scores_y_predict = calculate_zscore_anomalies(X_test['valueStdScaledVarForecast'].to_numpy())
    anomalies_errors_ae, z_scores_errors_ae = calculate_zscore_anomalies(errors_ae)
    anomalies_errors_se, z_scores_errors_se = calculate_zscore_anomalies(errors_se)
    
    anomalies_Y_test_mod, z_scores_Y_test_mod = calculate_modified_zscore_anomalies(X_test['valueStdScaled'].to_numpy())
    anomalies_y_predict_mod, z_scores_y_predict_mod = calculate_modified_zscore_anomalies(X_test['valueStdScaledVarForecast'].to_numpy())
    anomalies_errors_ae_mod, z_scores_errors_ae_mod = calculate_modified_zscore_anomalies(errors_ae)
    anomalies_errors_se_mod, z_scores_errors_se_mod = calculate_modified_zscore_anomalies(errors_se)

    aic = calculate_aic(len(X_test['TimeLatencyStdScaled']), 
                        mean_squared_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast']),
                        2)
    
    one_step_timeLatency = {
        'r2_score': r2_score(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast']),
        'mae': mean_absolute_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast']),
        'mape': mean_absolute_percentage_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast']),
        'mse': mean_squared_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast']),
        'pcc': np.corrcoef(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast'])[0,1],
        'aic': aic
    }

    aic = calculate_aic(len(X_test['valueStdScaled']), 
                        mean_squared_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast']),
                        2)
    
    one_step_throughput = {
        'r2_score': r2_score(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast']),
        'mae': mean_absolute_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast']),
        'mape': mean_absolute_percentage_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast']),
        'mse': mean_squared_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast']),
        'pcc': np.corrcoef(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast'])[0,1],
        'aic': aic
    }

    fig = plt.figure(figsize=(20,15))
    plt.title("Predict Anomalies T=" + str(T) + " with predict 1 on "+ str(model_exp))
    plt.plot(X_test['valueStdScaledVarForecast'].to_numpy(),label="Predict 1-step Forecast", alpha=0.6, c='red', linewidth=3)
    plt.plot(X_test['valueStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='black')
    plt.scatter(np.where(anomalies_ae==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_ae==True)], 
                alpha=0.8, color='green', s=350, label="3-Sigma Anomalies AE")
    plt.scatter(np.where(anomalies_se==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_se==True)], 
                alpha=0.8, color='magenta', s=300, label = "3-Sigma Anomalies SE")
    plt.scatter(np.where(anomalies_errors_ae==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_errors_ae==True)], 
                alpha=0.8, color='blue', s=250, label = "Z-score Anomalies AE")
    plt.scatter(np.where(anomalies_errors_se==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_errors_se==True)], 
                alpha=0.8, color='cyan', s=200, label = "Z-score Anomalies SE")
    plt.scatter(np.where(anomalies_errors_ae_mod==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_errors_ae_mod==True)], 
                alpha=0.8, color='lightgreen', s=150, label = "Modified Z-score Anomalies AE")
    plt.scatter(np.where(anomalies_errors_se_mod==True), X_test['valueStdScaledVarForecast'].to_numpy()[np.where(anomalies_errors_se_mod==True)], 
                alpha=0.8, color='orange', s=50, label = "Modified Z-score Anomalies SE")    
    plt.legend()    
    figName = f"Y_predict-1-step-anomalies-T_{T}-trial_{trial_fname}.png"
    plt.savefig(figName, transparent=False);
    
    n_step_metrics = {}    
    for predict_ahead in [5, 10, 15, 30, 60, 90, 120]:
        y_pred_n_steps = generate_nsteps_forecasts(X_test_2.to_numpy(), results_VAR, T, predict_ahead)
        
        X2 = np.array(y_pred_n_steps).reshape(-1,2)
        XX2 = pd.DataFrame(X2, columns=['valueStdScaled', 'TimeLatencyStdScaled'])    
        shape_dif = X_test.shape[0] - XX2.shape[0] 
        XX2_new = None
        if shape_dif > 0:
            XX2_new = expand_dataframe_with_nsteps(XX2, shape_dif)
        elif shape_dif < 0:
            XX2_new = XX2[0:X_test.shape[0]]
        else:
            XX2_new = XX2.copy()   
            
        XX2_new.set_index(X_test.index, inplace=True)
        # These are the predictions predict_ahead steps at a time
        X_test['valueStdScaledVarForecast2'] = XX2_new['valueStdScaled']
        X_test['TimeLatencyStdScaledVarForecast2'] = XX2_new['TimeLatencyStdScaled']        

        errors_ae2 = calculate_absolute_prediction_errors(X_test['valueStdScaled'].to_numpy(), X_test['valueStdScaledVarForecast2'].to_numpy())
        anomalies_ae2 = calculate_3sigma_anomalies(errors_ae2)        
        errors_se2 = calculate_squared_prediction_errors(X_test['valueStdScaled'].to_numpy(), X_test['valueStdScaledVarForecast2'].to_numpy())
        anomalies_se2 = calculate_3sigma_anomalies(errors_se2)
        anomalies_y_pred_nsteps_mod, z_scores_y_pred_nsteps_mod = calculate_modified_zscore_anomalies(X_test['valueStdScaledVarForecast2'].to_numpy())
        anomalies_errors_ae2_mod, z_scores_errors_ae2_mod = calculate_modified_zscore_anomalies(errors_ae2)
        anomalies_errors_se2_mod, z_scores_errors_se2_mod = calculate_modified_zscore_anomalies(errors_se2)
        
        anomalies_3sigma_y_pred_nsteps = calculate_3sigma_anomalies(X_test['valueStdScaledVarForecast2'].to_numpy())
        anomalies_y_pred_nsteps, z_scores_y_pred_nsteps = calculate_zscore_anomalies(X_test['valueStdScaledVarForecast2'].to_numpy())
        anomalies_errors_ae2, z_scores_errors_ae2 = calculate_zscore_anomalies(errors_ae2)
        anomalies_errors_se2, z_scores_errors_se2 = calculate_zscore_anomalies(errors_se2)

        crt_step_timeLatency = f'predict_ahead_{predict_ahead})_timeLatency'
        crt_step_throughput = f'predict_ahead_{predict_ahead})_throughput'

        aic = calculate_aic(len(X_test['TimeLatencyStdScaled']), 
                            mean_squared_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2']),
                            2)
        
        n_step_metrics[crt_step_timeLatency] = {'r2_nStep': r2_score(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2']),
                                                'mae_nStep': mean_absolute_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2']),
                                                'mape_nStep': mean_absolute_percentage_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2']),
                                                'mse_nStep': mean_squared_error(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2']),
                                                'pcc_nStep': np.corrcoef(X_test['TimeLatencyStdScaled'], X_test['TimeLatencyStdScaledVarForecast2'])[0,1],
                                                'aic_nStep': aic}

        aic = calculate_aic(len(X_test['valueStdScaled']), 
                            mean_squared_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2']),
                            2)

        n_step_metrics[crt_step_throughput] = {'r2_nStep': r2_score(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2']),
                                               'mae_nStep': mean_absolute_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2']),
                                                'mape_nStep': mean_absolute_percentage_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2']),
                                                'mse_nStep': mean_squared_error(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2']),
                                                'pcc_nStep': np.corrcoef(X_test['valueStdScaled'], X_test['valueStdScaledVarForecast2'])[0,1],
                                                'aic_nStep': aic}
        
        fig = plt.figure(figsize=(20,15))        
        plt.title("Predict Anomalies T=" + str(T) + " with predict " + str(predict_ahead) + " on " + str(model_exp))
        plt.plot(X_test['valueStdScaledVarForecast2'].to_numpy(),label="Predict " + str(predict_ahead) + "-step Forecast", alpha=0.6, c='red', linewidth=3)
        plt.plot(X_test['valueStdScaled'].to_numpy(),label="Original Data", alpha=0.6, c='black')
        plt.scatter(np.where(anomalies_ae2==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_ae2==True)], alpha=0.8, color='green', s=350, label="Anomalies AE")
        plt.scatter(np.where(anomalies_se2==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_se2==True)], alpha=0.8, color='magenta', s=300, label = "Anomalies SE")
        plt.scatter(np.where(anomalies_errors_ae2==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_errors_ae2==True)], alpha=0.8, color='blue', s=250, label = "Z-score Anomalies AE")
        plt.scatter(np.where(anomalies_errors_se2==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_errors_se2==True)], alpha=0.8, color='cyan', s=200, label = "Z-score Anomalies SE")
        plt.scatter(np.where(anomalies_errors_ae2_mod==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_errors_ae2_mod==True)], alpha=0.8, color='lime', s=150, label = "Modified Z-score Anomalies AE")
        plt.scatter(np.where(anomalies_errors_se2_mod==True), X_test['valueStdScaledVarForecast2'].to_numpy()[np.where(anomalies_errors_se2_mod==True)], alpha=0.8, color='orange', s=50, label = "Modified Z-score Anomalies SE")        
        plt.legend();    
        figName = f"Y-predict-anomalies-step-{predict_ahead}-with-T_{T}-trial-{trial_fname}.png"
        plt.savefig(figName, transparent=False)        

    results[trial_fname] = {"one_step_throughput" : one_step_throughput,
                            "one_step_time_latency" : one_step_timeLatency,
                            "n_step_metric": n_step_metrics}
    

In [ ]:
results

In [ ]:
for item in results.keys():
    print(results[item]['one_step_throughput'])